## Notebook para processar dados dos anos do enem e processa-los

In [14]:
import fitz
from typing import List,Dict
import re
import json

## Para o ano de 2009

In [2]:
reader = fitz.open('data/raw/2019/caderno-de-questoes-1-dia-caderno-1-azul-aplicacao-regular.pdf')
# for line in reader[1].get_text().split('\n'):
#     print(line)
print(reader[1].get_text())

LC - 1º dia | Caderno 1 - AZUL - Página 2
LINGUAGENS, CÓDIGOS  
E SUAS TECNOLOGIAS
Questões de 01 a 45
Questões de 01 a 05 (opção inglês)
Questão 01 
5 Ways Pets Can Improve Your Health
A pet is certainly a great friend. After a difficult day, 
pet owners quite literally feel the love.
In fact, for nearly 25 years, research has shown that 
living with pets provides certain health benefits. Pets help 
lower blood pressure and lessen anxiety. They boost our 
immunity. They can even help you get dates.
Allergy Fighters: A growing number of studies have 
suggested that kids growing up in a home with “furred 
animals” will have less risk of allergies and asthma.
Date Magnets: Dogs are great for making love 
connections. Forget Internet matchmaking — a dog is a 
natural conversation starter.
Dogs for the Aged: Walking a dog or just caring for 
a pet — for elderly people who are able — can provide 
exercise and companionship.
Good for Mind and Soul: Like any enjoyable activity, 
playing with 

In [3]:
DAY_1_QUESTIONS = ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","49","50","51","52","53","54","55","56","57","58","59","60","61","62","63","64","65","66","67","68","69","70","71","72","73","74","75","76","77","78","79","80","81","82","83","84","85","86","87","88","89","90","91","92","93","94","95","96","97","98","99","100","101","102","103","104","105","106","107","108","109","110","111","112","113","114","115","116","117","118","119","120","121","122","123","124","125","126","127","128","129","130","131","132","133","134","135","136","137","138","139","140","141","142","143","144","145","146","147","148","149","150","151","152","153","154","155","156","157","158","159","160","161","162","163","164","165","166","167","168","169","170","171","172","173","174","175","176","177","178","179","180"]

In [64]:
def concat_lines_questions(
    text_page: str
):
    questions = []
    
    aux = []
    start_question= False
    for line in text_page.split('\n'):
        
        if line.lower().startswith("questão "):
            start_question = True
            if len(aux) != 0:
                questions.append(aux)
            aux = [line]
        elif start_question:
            aux.append(line)

    if len(aux) != 0:
        questions.append(aux)

    return questions

def concat_numbers(
    questions: List[List[str]]
):
    new_questions = []
    for question in questions:
        if question[0].lower().startswith("questão ") and question[1].strip().isnumeric():
            new_question = [question[0] + " "+ question[1].strip()] + question[2:]
            new_questions.append(new_question)
        else:
            new_questions.append(question)
    return new_questions

def identify_alternatives(
    questions: List[List[str]]
):
    new_questions = []
    
    for question in questions:
        
        alternatives = []
        
        aux = ""
        index_alternatives = None
        for index in range(len(question)-1,-1,-1):
            
            aux = question[index] + " " + aux
            if question[index][0] in ["A","B","C","D","E","F","G"]:
                alternatives = [aux] + alternatives
                aux = ""
            if question[index][0] == "A":
                index_alternatives = index
                break
        
        new_questions.append({"description":question[:index_alternatives], "alternatives": alternatives})
        
    return new_questions
    
    
def ignore_lines(
    questions: List[List[str]]
):
    
    regexes = ["\*[A-Z0-9]*\*","[A-Z] *"]
    values = ['',' ']
    some = [
        'The English Enlightenment',
        'SILVA','São Paulo: Livraria',
        '2006. p. 169 (adaptado)',
        'LINGUAGENS, CÓDIGOS',
        'E SUAS TECNOLOGIAS',
        ' 01 a 05 (opção espanhol)',
        'de 01 a 45'
    ]
    
    new_questions = []
    for question in questions:
        question = list(filter(lambda x: not(any([re.fullmatch(r,x) for r in regexes])), question))
        question = list(filter(lambda x: not(any([v == x for v in values])), question))
        question = list(filter(lambda x: not(any([v in x for v in some])), question))
        new_questions.append(question)
    
    return new_questions
              
    
def concat_description(
   questions: List[Dict]
):
    new_questions = []
    
    for question in questions:
        
        aux = question['description'][0].strip() + ")"
        for line in question['description'][1:]:
            if aux[-1] == ".":
                aux += "\n"
            else:
                aux += " "
            aux += line
        
        question['description'] = aux
        new_questions.append(question)
    return new_questions

def filter_questions(
    questions: List[Dict]
):
    pass
    

def process_exam(
    path_exam: str
):
    reader = fitz.open(path_exam)
    
    questions = []
    for index, page in enumerate(reader):
        questions.extend(concat_lines_questions(page.get_text()))
    
    questions = ignore_lines(questions)
    questions = concat_numbers(questions)
    questions = identify_alternatives(questions)
    questions = concat_description(questions)
    
    return questions
    
    
def process_all_exams(
    exams: List[Dict]
):
    for year, exams in exams.items():
        day1_questions = process_exam(f"data/raw/{year}/{exams['day1']}")
        day2_questions = process_exam(f"data/raw/{year}/{exams['day2']}")
        
        all_questions = day1_questions + day2_questions
        
        print(year,len(all_questions))
        with open(f"data/cleaned/{year}.json","w") as f:
            f.write(json.dumps(all_questions,indent=2,ensure_ascii=False))

In [65]:
all_exams = {
    
    "2010" : { 
        "day1": "AZUL_Sabado_GAB.pdf", 
        "day2": "AZUL_Domingo_GAB.pdf"
    },
    "2011" : { 
        "day1": "01_AZUL_GAB.pdf", 
        "day2": "05_AMARELO_GAB.pdf"
    },
    "2012" : { 
        "day1": "4a6fb7236b8588c5e7bcd1c5f24132d0.pdf", 
        "day2": "8411c969b9bbfc9c08454fb1be37198e.pdf"
    },
    "2013" : { 
        "day1": "e0e555097eefaba6c44f690496b7314f.pdf", 
        "day2": "36b9b0bbcffef377b6b6d1fd70b851a4.pdf"
    },
    "2014" : { 
        "day1": "7e7c82295c41a2a540e7386829018b7a.pdf", 
        "day2": "9e65d0fb4bbcb4747dede4431d9e042d.pdf"
    },
    "2015" : { 
        "day1": "d13552b42f87140e48f9ce39688f5247.pdf", 
        "day2": "750c4a32e08306e4ba04c13c6152780d.pdf"
    },
    "2016" : { 
        "day1": "40698f20dfe44ed8745082cbf2907578.pdf", 
        "day2": "2eac4c7473cb5b18e11dea37f14ceacc.pdf"
    },
    "2017" : { 
        "day1": "b590e6afa19f6a68f66fd56579fd74af.pdf", 
        "day2": "6b7d8655361ff77316afd9d57b96ca8f.pdf"
    },
    "2018" : { 
        "day1": "66ad242a443e7dafcad23da8c0df4c4c.pdf", 
        "day2": "c5f7dfb0a30867e9c3ac732cfade9436.pdf"
    },
    "2019" : { 
        "day1": "caderno-de-questoes-1-dia-caderno-1-azul-aplicacao-regular.pdf", 
        "day2": "caderno-de-questoes--2-dia-caderno-5-amarelo-aplicacao-regular.pdf"
    },
    "2020" : { 
        "day1": "1-dia-caderno1-azul-prova.pdf", 
        "day2": "2-dia-caderno5-amarelo-prova.pdf"
    },
    "2021" : { 
        "day1": "caderno-azul.pdf", 
        "day2": "caderno-amarelo-d2.pdf"
    },
    "2022" : { 
        "day1": "1-dia-caderno-1-azul-enem-2022.pdf", 
        "day2": "2-dia-caderno-7-azul-enem-2022.pdf"
    }
}

process_all_exams(all_exams)

2010 186
2011 185
2012 185
2013 185
2014 186
2015 186
2016 185
2017 185
2018 185
2019 185
2020 186
2021 185
2022 185


In [40]:
# print(process_exam("data/raw/2019/caderno-de-questoes-1-dia-caderno-1-azul-aplicacao-regular.pdf")[0].keys())
# for q in process_exam("data/raw/2019/caderno-de-questoes--2-dia-caderno-5-amarelo-aplicacao-regular.pdf"):
#     for a in q['description']:
#         print(a)
#     print()
n = 55
questao = process_exam("data/raw/2019/caderno-de-questoes-1-dia-caderno-1-azul-aplicacao-regular.pdf")[n]
print(questao['description'])
print()
for a in questao['alternatives']:
    print(a)

Questão 51) Em sentido geral e fundamental, Direito é a técnica  da coexistência humana, isto é, a técnica voltada a   tornar possível a coexistência dos homens. Como  técnica, o Direito se concretiza em um conjunto de regras  (que, nesse caso, são leis ou normas); e tais regras  têm por objeto o comportamento intersubjetivo, isto é,   o comportamento recíproco dos homens entre si.
ABBAGNANO, N. Dicionário de Filosofia.  São Paulo: Martins Fontes, 2007.
O sentido geral e fundamental do Direito, conforme foi  destacado, refere-se à

A aplicação de códigos legais. 
B regulação do convívio social. 
C legitimação de decisões políticas. 
D mediação de conflitos econômicos. 
E representação da autoridade constituída. 


In [ ]:
process_exam("data/raw/2019/caderno-de-questoes-1-dia-caderno-1-azul-aplicacao-regular.pdf")